## Support Vector Machine - Regression

In [81]:
import numpy as np
from sklearn.svm import SVR
from sklearn import  model_selection
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import preprocessing
import pandas as pd
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
import sklearn.metrics as metrics
from joblib import dump, load

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)

In [82]:
df = pd.read_csv('imoveis.csv')

In [83]:
df.head()

,local,preco,metragem,quartos,garagens,banheiros,bairro
0,"Rua Maria Thereza Gonçalves, Umbara",399900,106,3.0,2.0,2,76
1,"Travessa Paulo Ribeiro, Sitio Cercado",765190,319,5.0,3.0,4,70
2,"Uberaba, Curitiba",2350000,500,5.0,7.0,5,75
3,"Rua Ângelo Greca, Atuba",737000,144,4.0,NaN,4,7
4,"Rua João Fonseca Mercer, Atuba",1140000,276,5.0,3.0,5,7


In [84]:
x = df.drop("preco", axis=1).drop("local", axis=1)
y = df["preco"]

In [85]:
scaler = StandardScaler()
scaler_preco = StandardScaler()

features = [['metragem', 'quartos', 'garagens', 'banheiros', 'bairro']]
for feature in features:
    x[feature] = scaler.fit_transform(x[feature])

y = scaler_preco.fit_transform(y.values.reshape(-1,1))

In [86]:
x.fillna(0, inplace=True)
Y =y
X = x.to_numpy()

In [87]:
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [88]:
regressor =  SVR()
regressor.fit(x_treinamento, y_treinamento)

SVR()

In [89]:
previsoes = regressor.predict(x_teste)

In [90]:
previsoes

array([-0.24833013,  0.14724304, -0.11105356, ..., -0.14890687,
        0.75976935, -0.10787999])

In [91]:
print("MAE:",scaler_preco.inverse_transform(metrics.mean_absolute_error(y_teste, previsoes).reshape(-1,1))[0][0])
#print("MSE:", scaler_preco.inverse_transform(metrics.mean_squared_error(y_teste, previsoes).reshape(-1,1))[0][0])
#print("MSLE:",scaler_preco.inverse_transform(metrics.mean_squared_log_error(y_teste, previsoes).reshape(-1,1))[0][0])
print("R2:", metrics.r2_score(y_teste, previsoes))

MAE: 1074390.2056818237
R2: 0.5485556898368904


In [92]:
scaler_preco.inverse_transform(previsoes.reshape(-1, 1))[:3].T

array([[664858.78348291, 962068.79857432, 768000.18864561]])

In [93]:
scaler_preco.inverse_transform(y_teste.reshape(-1, 1))[:3].T

array([[ 780000., 1400000.,  730000.]])

In [94]:
area = 247.6
quartos = 4
garagens = 4
banheiros = 3
bairro = 61 #Santa Felicidade
valores = scaler.transform([[area, quartos, garagens, banheiros, bairro]])
casa = regressor.predict(valores)

print("Preço estimado: R$", scaler_preco.inverse_transform(casa.reshape(-1,1))[0][0])

Preço estimado: R$ 883144.540298085


In [95]:
with open("models/SVM.mod", 'wb') as fo:  
    dump(regressor, fo)